In [59]:
import math
import pandas as pd
from enum import Enum

class MessageKind(Enum):
    MEASURE = "MEASURE"
    GENREQ = "GENREQ"
    GENRESP = "GENRESP"
    
class ReqResKind(Enum):
    READ = "READ"
    COMMAND = "COMMAND"
    UNSET = "UNSET"
    
class AtkStatus(Enum):
    UNMOD = "UNMOD"
    COMPR = "COMPR"
    BLOCK = "BLOCK"
    
class EvilState(Enum):
    INACTIVE = "INACTIVE"
    MIDDLE = "MIDDLE"
    FULL = "FULL"
    
evilStateToVar = {
    EvilState.INACTIVE.value: 1,
    EvilState.MIDDLE.value: 2,
    EvilState.FULL.value: 3
}
    
# Simulation time limit in seconds
simTimeLimit = 120
# Number of runs
numRuns = 1
# Connection index (0 for server[0] etc...)
connIndex = 0
# Discretization time in seconds
deltaT = 5


relPath = "../logs/"
evilFileName = "evilClient.app[" + connIndex.__str__() + "].csv"
cliFileName = "client[" + connIndex.__str__() + "].app[0].csv"
serFileName = "server[" + connIndex.__str__() + "].app[0].csv"


In [62]:
for i in range(numRuns):
    complRelPath = relPath + "run" + i.__str__() + "/"
    evilDf = pd.read_csv(complRelPath + evilFileName)
    evilDf = evilDf.drop("creationTime", axis=1)
    
    currT = 0
    MITMv = 0
    SRMv = 0
    UCv = 0
    while currT < simTimeLimit:
        evilWindowDf = evilDf[evilDf['timestamp'].between(currT, currT + deltaT, inclusive="left")]
        totNumWindow = evilWindowDf.shape[0]
        if not evilWindowDf.empty:
            MITMv = evilStateToVar[evilWindowDf.iloc[-1]["evilState"]]
            numComprMeas = evilWindowDf[((evilWindowDf["messageKind"] == MessageKind.MEASURE.value) | 
                                            (evilWindowDf["reqResKind"] == ReqResKind.READ.value)) & 
                                            (evilWindowDf["atkStatus"] == AtkStatus.COMPR.value)].shape[0]
            numComprCmds = evilWindowDf[(evilWindowDf["reqResKind"] == ReqResKind.COMMAND.value) & 
                                            (evilWindowDf["atkStatus"] == AtkStatus.COMPR.value)].shape[0]
            SRMv = max(math.trunc((numComprMeas/totNumWindow) * 4), SRMv)
            UCv = max(math.trunc((numComprCmds/totNumWindow) * 4), UCv)
            print(SRMv)
            
            
            
            
        currT += deltaT


0
0
0
0
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
